## Install wandb if it hasn't been installed on the system yet

In [ ]:
# only run this once
#! pip install wandb
! wandb login

In [ ]:
import copy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torchvision.io import read_image
from torchsummary import summary

from tqdm import tqdm
from datetime import datetime
from sklearn.model_selection import train_test_split

import wandb

from ece9603_project import lossFunctions

device = "cpu"
if torch.cuda.is_available():
    device="cuda"

In [ ]:
# As a small example, we can adjust this for future hyper parameter tuning
wandb_config = wandb.config = {
    "learning_rate": 0.001,
    "epochs": 10,
    "loss_function": "focal_tversky_loss"
}

wandb.init(project="breast-histopathology-classification",
           entity="ece9603_project",
           job_type="model_training",
           config=wandb_config)

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.info = df
        self.transform = transform

    def __len__(self):
        return len(self.info)

    def __getitem__(self, idx):
        path = self.info.imgPath.values[idx]
        label = self.info['class'].values[idx]
        image = read_image(path, mode=torchvision.io.image.ImageReadMode.RGB).float()

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
df= pd.read_csv("breastCancerDataframe.csv", index_col=0)
print(df.head())

patientIDs = df.patient.unique()
print("Number of Unique Patients: ", len(patientIDs))

patients_train, temp = train_test_split(patientIDs, test_size=0.3, random_state=42)
patients_val, patients_test = train_test_split(temp, test_size=0.5, random_state=42)

df_train = df.loc[df['patient'].isin(patients_train)]
print(df_train.head())
print("Number of Train Patients: ", df_train.patient.nunique())

df_val = df.loc[df['patient'].isin(patients_val)]
print(df_val.head())
print("Number of Validation Patients: ", df_val.patient.nunique())

df_test = df.loc[df['patient'].isin(patients_test)]
print(df_test.head())
print("Number of Test Patients: ", df_test.patient.nunique())

In [ ]:
BATCH_SIZE = 128

transform = transforms.Compose([
        #transforms.RandomRotation(45),
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_dataset = CustomImageDataset(df_train, transform=transform)
val_dataset = CustomImageDataset(df_val, transform=transform)
test_dataset = CustomImageDataset(df_test, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [ ]:
"""
model=models.resnet18(pretrained=True)
print(model)

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.5),

    nn.Linear(512, 256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(0.5),

    nn.Linear(256, 1))

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)
"""

In [ ]:
model = models.efficientnet_b0(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(1280, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),

    nn.Dropout(0.2),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),

    nn.Linear(256, 1))

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)
#summary(model, (3,224,224))

In [ ]:
# Calculate performance measures
def compute_performance(yhat, y, pos_cutoff):

    # First, get tp, tn, fp, fn
    tp = sum(np.logical_and(yhat >= pos_cutoff, y == 1).numpy())
    tn = sum(np.logical_and(yhat < pos_cutoff, y == 0).numpy())
    fp = sum(np.logical_and(yhat >= pos_cutoff, y == 0).numpy())
    fn = sum(np.logical_and(yhat < pos_cutoff, y == 1).numpy())

    print(f"tp: {tp} tn: {tn} fp: {fp} fn: {fn}")

    # Accuracy
    #acc = (tp + tn) / (tp + tn + fp + fn)

    # Precision
    # "Of the ones I labeled +, how many are actually +?"
    precision = tp / (tp + fp)

    # Recall
    # "Of all the + in the data, how many do I correctly label?"
    recall = tp / (tp + fn)

    # Sensitivity
    # "Of all the + in the data, how many do I correctly label?"
    sensitivity = recall

    # Specificity
    # "Of all the - in the data, how many do I correctly label?"
    specificity = tn / (fp + tn)

    balanced_acc = 0.5*(sensitivity+specificity)
    #fMeasure =  2*((precision*recall)/(precision+recall))

    # Print results
    print("Balanced Accuracy: ", balanced_acc," Specificity: ",specificity,
          " Sensitivity: ",sensitivity," Precision: ",precision,)
    # Log results to WandB
    wandb.log({"Balanced Accuracy": balanced_acc,
               "Specificity": specificity,
               "Sensitivity": sensitivity,
               "Precision": precision},
              commit=False)

#fMeasure =  2*((performance[1]*performance[2])/(performance[1]+performance[2]))

def train(model, dataloader_train, dataloader_val, device='cpu', epochs=10, early_stop=2, lr=0.001, 
          loss_function='bce_with_logits_loss', verbose=True):

    opt = torch.optim.Adam(model.classifier.parameters(), lr=lr)
    #opt = torch.optim.Adam(model.fc.parameters(), lr=lr)
    criterion = lossFunctions.getLossFunction(loss_function)

    model.to(device)

    lowest_val_loss, train_loss = np.inf, 0
    lowest_val_epoch = 0
    epochs_wo_improvement = 0
    best_model = copy.deepcopy(model.state_dict())
    train_losses, val_losses=[], []

    for e in range(epochs):
        epoch_train_loss = 0
        epoch_val_loss = 0

        model.train()

        for inputs, targets in tqdm(dataloader_train):

            inputs, targets = inputs.to(device), targets.to(device)

            model.zero_grad(set_to_none=True)

            output = model.forward(inputs)
            loss = criterion(output.squeeze(), targets.float())
            loss.backward()
            opt.step()

            epoch_train_loss+=loss

        epoch_train_loss = epoch_train_loss.item()/((len(dataloader_train.dataset)%BATCH_SIZE)*BATCH_SIZE)

        train_losses.append(epoch_train_loss)

        #VALIDATION

        model.eval()
        val_preds, val_targets_list = [], []

        with torch.no_grad():
            for val_inputs, val_targets in tqdm(dataloader_val):

                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)

                model.zero_grad(set_to_none=True)

                val_output = model.forward(val_inputs).squeeze()
                val_preds+=val_output
                val_targets_list+=val_targets

                epoch_val_loss += criterion(val_output, val_targets.float())

            epoch_val_loss = epoch_val_loss.item()/((len(dataloader_val.dataset)%BATCH_SIZE)*BATCH_SIZE)
            val_losses.append(epoch_val_loss)

            compute_performance(torch.sigmoid(torch.Tensor(val_preds)), torch.Tensor(val_targets_list), 0.5)

        if epoch_val_loss <= lowest_val_loss:
            best_model = copy.deepcopy(model.state_dict())
            lowest_val_loss = epoch_val_loss
            train_loss=epoch_train_loss
            lowest_val_epoch=e
            epochs_wo_improvement=0
        else:
            epochs_wo_improvement+=1

        if verbose:
            print("Epoch: {}/{}...".format(e, epochs), "Loss: {:.4f}...".format(epoch_train_loss), "Val Loss: {:.4f}".format(epoch_val_loss),)
        # Log to wandb project
        wandb.log({"training_loss": epoch_train_loss,
                   "validation_loss": epoch_val_loss})
            
        #early stopping
        if epochs_wo_improvement>=early_stop:
            if verbose:
                print("Early Stop no improvement in validation loss in "+str(early_stop)+" validation steps")
            break

    if verbose:
        print("\nLowest Validation Loss: "+str(lowest_val_loss)+" at epoch "+str(lowest_val_epoch)+'\n')

    model.load_state_dict(best_model)
    # Record model to wandb
    wandb.watch(model)

    run_ID = datetime.now().strftime("%Y-%m-%d_%H-%M")
    torch.save({'model_state_dict': best_model}, './BestModels/'+str(run_ID)+'_E_'+str(lowest_val_epoch)+'_TL_'+str(round(train_loss, 4))+'_VL_'+str(round(lowest_val_loss, 4))+'.pt')

    model.to("cpu")
    torch.cuda.empty_cache()

    return model, train_losses, val_losses

In [ ]:
torch.backends.cudnn.benchmark = True
model, train_losses, val_losses = train(model, train_dataloader, val_dataloader, device=device, loss_function=wandb_config['loss_function'])
# Done this training run
wandb.finish()

In [ ]:
model.to(device)
model.eval()
test_preds, test_targets_list = [], []
test_loss = 0

with torch.no_grad():
    criterion = nn.BCEWithLogitsLoss()

    for test_inputs, test_targets in tqdm(test_dataloader):

        test_inputs, test_targets = test_inputs.to(device), test_targets.to(device)

        model.zero_grad(set_to_none=True)

        test_output = model.forward(test_inputs).squeeze()
        test_preds+=test_output
        test_targets_list+=test_targets

        test_loss += criterion(test_output, test_targets.float())

    print("Test Loss: ", test_loss.item()/((len(test_dataloader.dataset)%BATCH_SIZE)*BATCH_SIZE))

    compute_performance(torch.sigmoid(torch.Tensor(test_preds)), torch.Tensor(test_targets_list), 0.5)

In [ ]:
print("Percent Neg: ", (df_train['class'].to_list()).count(0)/len(df_train))
print("Percent Pos: ", (df_train['class'].to_list()).count(1)/len(df_train))

df_train_neg = df_train.loc[df_train['class']==0]
df_train_pos = df_train.loc[df_train['class']==1]
print("Neg Count: ", len(df_train_neg), "Pos Count: ",len(df_train_pos))

#split it into 3 since i would rater the model have an over represented positive class so the model
#predicts pos more than neg since identifying pos is more important
df_train1, temp = train_test_split(df_train_neg, train_size=1/3, random_state=42)
df_train2, df_train3 = train_test_split(temp, train_size=1/2, random_state=42)
df_train1 = pd.concat([df_train1, df_train_pos], ignore_index=True)
df_train2 = pd.concat([df_train2, df_train_pos], ignore_index=True)
df_train3 = pd.concat([df_train3, df_train_pos], ignore_index=True)


fig, ax = plt.subplots(1,4,figsize=(25,5))
sns.countplot(df_train['class'], ax=ax[0], palette="Reds")
ax[0].set_title("Original Train Data")
sns.countplot(df_train1['class'], ax=ax[1], palette="Blues")
ax[1].set_title("Balanced Train Data 1")
sns.countplot(df_train2['class'], ax=ax[2], palette="Greens")
ax[2].set_title("Balanced Train Data 2")
sns.countplot(df_train3['class'], ax=ax[3], palette="Oranges")
ax[3].set_title("Balanced Train Data 3")

print("\nClass Percentages After Splitting")
print("Percent Neg Train1: ", (df_train1['class'].to_list()).count(0)/len(df_train1), "Percent Pos Train 1: ", (df_train1['class'].to_list()).count(1)/len(df_train1))
print("Percent Neg Train2: ", (df_train2['class'].to_list()).count(0)/len(df_train2), "Percent Pos Train 2: ", (df_train2['class'].to_list()).count(1)/len(df_train2))
print("Percent Neg Train3: ", (df_train3['class'].to_list()).count(0)/len(df_train3), "Percent Pos Train 3: ", (df_train3['class'].to_list()).count(1)/len(df_train3))

train_dataloaders = [DataLoader(CustomImageDataset(df_train1, transform=transform), batch_size=BATCH_SIZE, shuffle=True), DataLoader(CustomImageDataset(df_train2, transform=transform), batch_size=BATCH_SIZE, shuffle=True), DataLoader(CustomImageDataset(df_train3, transform=transform), batch_size=BATCH_SIZE, shuffle=True)]

In [ ]:
def init_models(dataloaders, load_save=False):
    if load_save:
        ensemble = [models.efficientnet_b0(pretrained=True) for _ in dataloaders]

        for model in ensemble:
            for param in model.parameters():
                param.requires_grad = False

            model.classifier = nn.Sequential(
                nn.Dropout(0.2),
                nn.Linear(1280, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(),

                nn.Dropout(0.2),
                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                nn.ReLU(),

                nn.Linear(256, 1))

            model.apply(init_weights)
    else:
        ensemble = [[models.efficientnet_b0(pretrained=True), data] for data in dataloaders]

        for model, _ in ensemble:
            for param in model.parameters():
                param.requires_grad = False

            model.classifier = nn.Sequential(
                nn.Dropout(0.2),
                nn.Linear(1280, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(),

                nn.Dropout(0.2),
                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                nn.ReLU(),

                nn.Linear(256, 1))

            model.apply(init_weights)
    return ensemble

In [ ]:
ensemble = init_models(dataloaders=train_dataloaders)
trained_ensemble = []

torch.backends.cudnn.benchmark = True
for mod, dataloader_train in ensemble:
    model, train_losses, val_losses = train(mod, dataloader_train, val_dataloader, early_stop=1, device=device)
    trained_ensemble.append(model)

In [ ]:
def ensemble_predict(models, dataloader_test, device='cpu'):
    for mod in models:
        mod.to(device)
        mod.eval()

    test_loss = 0
    test_preds, test_targets_list = [], []
    criterion = nn.BCELoss()

    with torch.no_grad():
        for test_inputs, test_targets in tqdm(dataloader_test):

            test_inputs, test_targets = test_inputs.to(device), test_targets.to(device)
            batch_output=[]

            for model in models:
                model.zero_grad(set_to_none=True)
                batch_output.append(torch.sigmoid(model.forward(test_inputs).squeeze()).cpu().numpy())

            #average models output
            batch_output = np.column_stack(batch_output)
            test_output = np.mean(batch_output, axis=1)

            test_preds = np.hstack((test_preds, test_output))
            test_targets_list+=test_targets
            test_loss += criterion(torch.tensor(test_output).to(device), test_targets.float())

        print("Test Loss: ", test_loss.item()/((len(dataloader_test.dataset)%BATCH_SIZE)*BATCH_SIZE))

        compute_performance(torch.Tensor(test_preds), torch.Tensor(test_targets_list), 0.5)

load_save = True

if load_save:
    checkpoint1 = torch.load('BestModels/2021-11-21_17-33_E_3_TL_0.0162_VL_0.01.pt')
    checkpoint2 = torch.load('BestModels/2021-11-21_17-41_E_0_TL_0.0187_VL_0.01.pt')
    checkpoint3 = torch.load('BestModels/2021-11-21_18-03_E_3_TL_0.0161_VL_0.0097.pt')

    trained_ensemble = init_models(dataloaders=train_dataloaders, load_save=load_save)

    trained_ensemble[0].load_state_dict(checkpoint1['model_state_dict'])
    trained_ensemble[1].load_state_dict(checkpoint2['model_state_dict'])
    trained_ensemble[2].load_state_dict(checkpoint3['model_state_dict'])

ensemble_predict(trained_ensemble, test_dataloader, device='cuda')